In [59]:
import numpy as np

In [60]:
timetable_constraints = {
  "nbr_of_days": 5,
  "nbr_of_periods_per_day": 4,
  "nbr_of_subjects": 3,
  "nbr_of_groups": 2,
  "periods_by_subject": [2, 2, 2],
  "subject_names": ["Maths", "Français", "Anglais"],
  "group_names": ["Groupe 1", "Groupe 2"],
}

In [61]:
class Timetable:
  def __init__(self, npArray, constraints):
    self.constraints = constraints
    # timetable as a 3d array ( group, day, period) => subject from npArray
    self.timetable = npArray.reshape(
      (constraints["nbr_of_groups"], constraints["nbr_of_days"], constraints["nbr_of_periods_per_day"])
    )
    pass

  def toNpArray(self):
    return self.timetable.flatten()

  def isValid(self):
    pass

  def fitness(self):
    # return 1 if it is a valid timetable
    # return 0 if it is totally not
    # return a value between 0 and 0.8 if it is partially valid
    # a valid timetable is a timetable that respects all the constraints
    # a partially valid timetable is a timetable that respects some of the constraints
    # a totally invalid timetable is a timetable that respects none of the constraints
    # the value returned is the percentage of constraints that are respected

    respected_constraints = 0
    total_constraints = 0

    for group in self.timetable:
      subject_periods_in_table = np.zeros(self.constraints["nbr_of_subjects"])
      for day in group :
        for period in day:
          if period == -1: continue
          subject_periods_in_table[period]+=1
      respected_constraints += np.sum(subject_periods_in_table == self.constraints["periods_by_subject"])
      total_constraints += self.constraints["nbr_of_subjects"]
      
    if respected_constraints == total_constraints: return 1
    return 0.75*respected_constraints/total_constraints + 0.000001

  def plot(self):
    pass

  def print(self):
    print(self.fitness())
    i=0
    for group in self.timetable:
      i+=1
      print("Group : "+str(i))
      print(group)

In [62]:
import pygad

In [63]:
def fitness_func(ga_instance, solution, solution_idx):
    timeTable = Timetable(solution, timetable_constraints)
    fitness = timeTable.fitness()
    return fitness

In [64]:
fitness_function = fitness_func

num_generations = 100
num_parents_mating = 200

sol_per_pop = 10000
gene_space = [ 
              { 
                "low" : 0, 
                "high": timetable_constraints["nbr_of_subjects"],
                "step": 1
              } 
              for _ in range(timetable_constraints["nbr_of_groups"] * timetable_constraints["nbr_of_days"] * timetable_constraints["nbr_of_periods_per_day"])
             ]
num_genes = timetable_constraints["nbr_of_days"] * timetable_constraints["nbr_of_periods_per_day"] * timetable_constraints["nbr_of_groups"]
gene_type = [int for _ in range(num_genes)]

parent_selection_type = "sus"
keep_parents = 1

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 20

In [65]:
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       gene_space=gene_space,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       gene_type=gene_type,
                       num_genes=num_genes
                      )
                       
ga_instance.run()

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

Parameters of the best solution : [0 1 1 1 1 2 0 1 1 1 1 1 1 1 1 1 1 1 1 2 1 0 2 1 2 2 1 1 1 2 1 1 2 2 1 2 1
 0 1 1]
Fitness value of the best solution = 0.500001


In [ ]:
timetable = Timetable(solution, timetable_constraints)
timetable.print()

0.500001
Group : 1
[[0 1 1 1]
 [1 2 0 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 2]]
Group : 2
[[1 0 2 1]
 [2 2 1 1]
 [1 2 1 1]
 [2 2 1 2]
 [1 0 1 1]]
